In [189]:
"""
File for modelling full sampling procedure, with rotation rate, voltage fitting, profiling speed, etc.

"""

import numpy as np
import pandas as pd
from scipy.io import netcdf
from scipy.io import loadmat, savemat
from scipy import signal
from datetime import datetime
import matplotlib.pyplot as plt
import random as rand
import pyIGRF
from src.spectral_processing import process_files





%load_ext autoreload
%autoreload 2

ModuleNotFoundError: No module named 'src'

In [165]:
# Step 1 Pick a wave spectra

#fname = "/Volumes/TFO-exFAT-1/TFO/LCDRI/CDIPwaverider229/229p1_d01.nc";
fname = "/Users/jamesstadler/Documents/UW/TFO/Data/LCDRI/229p1_d01.nc"
[times, f, spectra] = load_CDIP(fname)
test_spectra = np.mean(spectra[150:160, :], 0)

In [166]:
# Step 2: Make a record of x-y-z positions of float

def apex_sampling_grid(start_z, end_z, prof_speed, u_prof, v_prof, sample_rate=1):
    """
    Inputs:
        start_z: Initial depth

        end_z: Final Depth

        prof_speed: Profiling spped (m/s)

        Sample_rate: Sampling frequency (defauly 1Hz for Em-APEX float)
        
    Outputs:
        
    """
    t_length = (end_z-start_z)/prof_speed
    num_samples = int(t_length*sample_rate)
    z_range = np.linspace(start_z, end_z, num_samples)
    t_array = np.linspace(0, t_length, num_samples)

    #Get the z-position
    em_z = t_array*prof_speed+start_z

    #Get the x-position
    em_x, em_y = get_xy_position(u_prof, v_prof)
    
    return(em_x, em_y, em_z)

def get_xy_position(u_prof, v_prof):
    """
        Returns x-y-position of float given some horizontal-velocity-profile
    """
    em_x = np.zeros(len(u_prof))
    em_y = np.zeros(len(v_prof))
    for i in range(len(u_prof)):
        em_x[i] = np.trapz(u_prof[:i])
        em_y[i] = np.trapz(v_prof[:i])
        
    return(em_x, em_y)
        

In [167]:
# Step 2: Build a wave based timeseries of u. Add to it the mean u profile

def build_u_timeseries(em_x, em_y, em_z, input_spectra, f, theta_spec, n_iter):

    fs = 1
    u_store = np.zeros((n_iter, len(em_x)))
    v_store = np.zeros((n_iter, len(em_x)))

    t_range = np.linspace(0, len(em_x), len(em_x))
    #Number of iterations to run
    #zeta_store = np.zeros((n_iter, len(t_range)))
    t = 0
    x = 0
    for jj in range(0, n_iter):
        for i in range(0, len(f-1)):#:#len(f)-1:
            freq = f[i]
            if i == 0:
                df = f[1]-f[0]
            elif i == len(f)-1:
                df = f[i]-f[i-1]
            else:
                df = (f[i+1]-f[i-1])/2
            omega = 2*np.pi*freq;
            k = np.square(omega)/9.8
            
            #This is assuming theta is angle CCW from east
            kx = k * np.cos(theta_spec[i])
            ky = k * np.sin(theta_spec[i])
            
            #Is there a 2 in here or not?
            a = np.sqrt(input_spectra[i]*df/2) # Is this the right conversion to wave amplitude? 

            #Randomize phase
            phi = rand.random()*2*np.pi

     

            u = a*omega*np.cos(kx*em_x-omega*t_range + phi)*np.exp(-k*em_z)
            v = a*omega*np.cos(ky*em_y-omega*t_range + phi)*np.exp(-k*em_z)
            
            #zeta = a*np.cos(k*x-omega*t_range + phi)
            #print(u)
            u_store[jj, :] = u_store[jj, :] + u
            v_store[jj, :] = v_store[jj, :] + v
    #how do we choose the amplitude for each frequency?



    ## Add white noise
    
    #First generate a white noise with the std of an EM-APEX float white noise situation
    #TO DO: Check if "uncertainty of 0.8-1.5 cm/s" means that's 1 Std or rms or what
    mean = 0
    std = 0.01 
    num_samples = len(u_store[:, 0])*len(u_store[0, :])
    rand_samples = 0.008*np.random.normal(loc = 0, scale = 1, size = num_samples)

    rand_samples = rand_samples.reshape((len(u_store[:, 0]), len(u_store[0, :])))
    u_noise = u_store + rand_samples
    v_noise = v_store + rand_samples
   

    return(u_store, v_store)



In [168]:
#Step 3 transform into voltages in x-y directions:




#Step 4 rotate each channel's position in time
#initial phase offset of channel 1
def get_channel_angs(t_length, rotation_rate, phi=0):

    ch1_angs = np.linspace(0, int(t_length), int(t_length))*rotation_rate*2*np.pi+phi
    ch2_angs = ch1_angs +np.pi/2 #Add 90deg for channel2
                           
    return(ch1_angs, ch2_angs)

In [185]:


alpha2 = 1.95
alpha1 = alpha2 - np.pi/2
#Run it all

#Define some input conditions
start_z = 0
end_z = 100
prof_speed = 0.1
sample_rate = 1
rotation_rate = 0.1

#Define some mean velocitiy profile
t_length = (end_z-start_z)/prof_speed
num_samples = int(t_length*sample_rate)
z_range = np.linspace(start_z, end_z, num_samples)
u_prof = 0.2*np.ones(int(t_length))#np.cos(z_range*2*np.pi/np.max(z_range))
v_prof = -0.03*np.ones(int(t_length))#np.cos(z_range*2*np.pi/np.max(z_range))


#build grid
[em_x, em_y, em_z] = apex_sampling_grid(start_z, end_z, prof_speed, u_prof, v_prof)


#Lets make a fake theta_spec for now:
#Send all the waves to the southeast.
theta_spec = np.ones(len(f))*-1*np.pi/4

#Number of simulations to run
n_iter = 1
[u_store, v_store] = build_u_timeseries(em_x, em_y, em_z, test_spectra, f, theta_spec, n_iter)

#Add the mean profile
u_store = u_store + u_prof
v_store = v_store + v_prof

#u_store = u_prof
#v_store = v_prof
#Step 3 get voltages
avg_lat = 33.474764
avg_lon = -119.288591

[Bx, By, Bz, _] = pyIGRF.calculate.igrf12syn(2017.221917808219, 1, 0, avg_lat, avg_lon)
fz=-np.nanmean(Bz);
esep1 = (8+5/8)*0.0254 # m
esep2 = (8+5/8)*0.0254 # m
c1 = 0.5 
Vx = fz*v_store*-1*esep1*(1+c1)/1000
Vy = -fz*u_store*-1*esep1*(1+c1)/1000


#Step 4 get the angular orientation of the float and convert voltages channel 1 and channel 2 components
ch1_angs, ch2_angs = get_channel_angs(t_length, rotation_rate)

#Rotate from magnetometer coordinates
Vxr = Vx*np.cos(alpha1)+Vy*np.sin(alpha1)
Vyr = Vx*np.sin(alpha1)-Vy*np.cos(alpha1)

E1 = np.cos(ch1_angs)*Vxr - np.sin(ch1_angs)*Vyr
E2 = -np.cos(ch1_angs)*Vyr - np.sin(ch1_angs)*Vxr


#Step 5 add random low frequency offset
xs = np.linspace(0, int(t_length), int(t_length))
offset = -200-.1*xs + 8*(10**-9)*(xs**2) - 8*(10**-8)*(xs**3)
E1 = E1 + offset
E2 = E1 + offset

#Angles are in magnetometer coordinates
HX = np.cos(ch1_angs)
HY = np.sin(ch1_angs)

#Now step 6, save it as fake EM data, and run it through the normal pipeline

#Need to save lat-lon data, Also time data

uxt_starttime = 1491226592.2198906
uxt = uxt_starttime+xs
EFR = {"E1": np.expand_dims(E1, axis=0), "E2": np.expand_dims(E2, axis=0), "HX": np.expand_dims(HX, axis=0), "HY": np.expand_dims(HY, axis=0), "UXT":np.expand_dims(uxt, axis=0), "SEQNO":np.expand_dims(xs, axis=0)}

CTD = {"P": np.expand_dims(z_range, axis=0), "UXT":np.expand_dims(uxt, axis=0)}


GPS = {"LAT": np.array([[avg_lat]]), "LON": np.array([[avg_lon]]), "GPS_UXT": np.array([[uxt_starttime]])}
down=False
if prof_speed>0:
    down=True
    
if down:
    savemat("/Users/jamesstadler/Documents/UW/TFO/Data/Simulations/ema-6667g-0001-efr.mat", EFR)
    savemat("/Users/jamesstadler/Documents/UW/TFO/Data/Simulations/ema-6667g-0002-gps.mat", GPS)
    savemat("/Users/jamesstadler/Documents/UW/TFO/Data/Simulations/ema-6667g-0002-ctd.mat", CTD)

In [188]:
process_files("/Users/jamesstadler/Documents/UW/TFO/Data/Simulations/")


NameError: name 'process_files' is not defined

In [36]:
if __name__='__main__':
    """
    do everything
    """

SyntaxError: invalid syntax (433286248.py, line 1)

In [58]:
t_length

1000.0